<a href="https://colab.research.google.com/github/comicom/COVID-19/blob/master/test/folium/corona_folium_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set and Test dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
basePath = "/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430"

In [0]:
import os

# We need to get all file
datasetList = []
# Get datasetPath
for root,dirs,files in os.walk(basePath,topdown=False):
  for name in files:
    path = os.path.join(root,name)
    if len(path.split('/'))>7 and path.split('/')[7]=="coronavirusdataset_20200430" and path.endswith("csv"):
      datasetList.append(path)

In [4]:
print(len(datasetList)) # If > 0, then a PNG image was loaded
print(datasetList)

12
['/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/SeoulFloating.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/Region.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/Time.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/TimeAge.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/TimeGender.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/TimeProvince.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/Weather.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/SearchTrend.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/Case.csv', '/content/drive/My Drive/Colab Notebooks/contest/dataset/coronavirusdataset_20200430/PatientRo

In [5]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import folium
from folium import plugins

import json
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
coronaData = {}
for i in range(len(datasetList)):
  coronaData[datasetList[i].split('.')[-2].split('/')[-1]] = pd.read_csv(datasetList[i])
print(coronaData.keys())

dict_keys(['SeoulFloating', 'Region', 'Time', 'TimeAge', 'TimeGender', 'TimeProvince', 'Weather', 'SearchTrend', 'Case', 'PatientRoute', 'Policy', 'PatientInfo'])


In [7]:
display(coronaData['Region'].head())
display(coronaData['Region'].isnull().sum() / coronaData['Region'].shape[0])

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080


code                        0.0
province                    0.0
city                        0.0
latitude                    0.0
longitude                   0.0
elementary_school_count     0.0
kindergarten_count          0.0
university_count            0.0
academy_ratio               0.0
elderly_population_ratio    0.0
elderly_alone_ratio         0.0
nursing_home_count          0.0
dtype: float64

In [8]:
display(coronaData['PatientInfo'].head())
display(coronaData['PatientInfo'].isnull().sum() / coronaData['PatientInfo'].shape[0])

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released


patient_id            0.000000
global_num            0.328217
sex                   0.022727
birth_year            0.136068
age                   0.024793
country               0.000000
province              0.000000
city                  0.023318
disease               0.994687
infection_case        0.226682
infection_order       0.990850
infected_by           0.757379
contact_number        0.810213
symptom_onset_date    0.858028
confirmed_date        0.000885
released_date         0.608323
deceased_date         0.981700
state                 0.000000
dtype: float64

In [9]:
print(coronaData['Region'].columns)
print(coronaData['PatientInfo'].columns)

Index(['code', 'province', 'city', 'latitude', 'longitude',
       'elementary_school_count', 'kindergarten_count', 'university_count',
       'academy_ratio', 'elderly_population_ratio', 'elderly_alone_ratio',
       'nursing_home_count'],
      dtype='object')
Index(['patient_id', 'global_num', 'sex', 'birth_year', 'age', 'country',
       'province', 'city', 'disease', 'infection_case', 'infection_order',
       'infected_by', 'contact_number', 'symptom_onset_date', 'confirmed_date',
       'released_date', 'deceased_date', 'state'],
      dtype='object')


# Plotting a map: the number of regional confirmed patient

In [10]:
regional_patient = pd.merge(coronaData['PatientInfo'][['patient_id','confirmed_date','sex','age','province','city']],
                            coronaData['Region'][['province','city','latitude','longitude']], how='left',
                            on = ['province','city'])
display(regional_patient.head())
display(regional_patient.isnull().sum() / regional_patient.shape[0])

,patient_id,confirmed_date,sex,age,province,city,latitude,longitude
0,1000000001,2020-01-23,male,50s,Seoul,Gangseo-gu,37.551166,126.849506
1,1000000002,2020-01-30,male,30s,Seoul,Jungnang-gu,37.606832,127.092656
2,1000000003,2020-01-30,male,50s,Seoul,Jongno-gu,37.572999,126.979189
3,1000000004,2020-01-30,male,20s,Seoul,Mapo-gu,37.566283,126.901644
4,1000000005,2020-01-31,female,20s,Seoul,Seongbuk-gu,37.589562,127.016700


patient_id        0.000000
confirmed_date    0.000885
sex               0.022727
age               0.024793
province          0.000000
city              0.023318
latitude          0.039847
longitude         0.039847
dtype: float64

In [11]:
m = folium.Map([36, 128],width='75%', height='75%', zoom_start=8) 

plugins.Fullscreen(position='topright',  # Full screen
                   title='Click to Expand', 
                   title_cancel='Click to Exit', 
                   force_separate_button=True).add_to(m)
regional_count = np.array(regional_patient[['latitude','longitude']].dropna()) # drop missing values
plugins.MarkerCluster(regional_count).add_to(m)
plugins.MousePosition().add_to(m) ## you can easily get coordinates.
m

In [13]:
regional_patient_bytime = regional_patient\
.groupby(['confirmed_date','latitude','longitude'])['patient_id'].count().reset_index()\
.sort_values(by = 'confirmed_date')
display(regional_patient_bytime.head())

,confirmed_date,latitude,longitude,patient_id
0,2020-01-23,37.551166,126.849506,1
1,2020-01-26,37.658363,126.831961,1
2,2020-01-27,36.992293,127.112709,1
3,2020-01-30,37.566283,126.901644,1
4,2020-01-30,37.572999,126.979189,1


In [15]:
date_rng = pd.date_range(regional_patient_bytime['confirmed_date'].min(), regional_patient_bytime['confirmed_date'].max())
date_rng = [str(i)[:10] for i in date_rng]

all_date_region = pd.DataFrame({'confirmed_date' : np.repeat(date_rng,coronaData['Region'][['latitude','longitude']].drop_duplicates().shape[0]),
              'latitude' : np.tile(list(coronaData['Region']['latitude'].drop_duplicates()), len(date_rng)),
              'longitude' : np.tile(list(coronaData['Region']['longitude'].drop_duplicates()), len(date_rng))})

all_date_region = pd.merge(all_date_region, regional_patient_bytime, on = ['confirmed_date','latitude','longitude'], how = 'left').fillna(0)
all_date_region['cumsum'] = all_date_region.groupby(['latitude','longitude'])['patient_id'].cumsum()

all_date_region = all_date_region[all_date_region['cumsum'] != 0]

time_index, data = [], []

for date in all_date_region['confirmed_date'].unique():
    time_index += [date[6:].replace('-','/')]
    temp_list = all_date_region.query('confirmed_date == @date')[['latitude','longitude','cumsum']].values.tolist()
    data += [temp_list]
    del temp_list

print(data[:5], time_index[:5])

[[[37.551165999999995, 126.849506, 1.0]], [[37.551165999999995, 126.849506, 1.0]], [[37.551165999999995, 126.849506, 1.0]], [[37.551165999999995, 126.849506, 1.0], [37.658363, 126.831961, 1.0]], [[37.551165999999995, 126.849506, 1.0], [37.658363, 126.831961, 1.0], [36.992293, 127.11270900000001, 1.0]]] ['1/23', '1/24', '1/25', '1/26', '1/27']


In [0]:
def color(total):
    # Color range
    col_100  = "#ff9900"
    col_1000 = "#cc33ff"
    col_3000 = "#ff0000"
    over     = "#660000"
    if (total < 100):   
            rad = total/10
            color = col_100
    elif (total < 1000): 
            rad = min(total/10, 20)
            color = col_1000
    elif (total < 3000): 
            rad = min(total/10, 30)
            color = col_3000
    else: 
            rad = 35
            color = over
    return rad, color

In [0]:
legend_html = '''
        <div style="position: fixed; bottom: 300px; left: 50px; width: 160px; height: 110px; 
                    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;"
                    >&nbsp; <b>Legend</b> <br>
                    &nbsp; Confirmed < 100 &nbsp&nbsp&nbsp; 
                        <i class="fa fa-circle" style="font-size:14px;color:#ff9900"></i><br>
                    &nbsp; Confirmed < 1000 &nbsp; 
                        <i class="fa fa-circle" style="font-size:14px;color:#cc33ff"></i><br>
                    &nbsp; Confirmed < 3000 &nbsp; 
                        <i class="fa fa-circle" style="font-size:14px;color:#ff0000"></i><br>
                    &nbsp; Confirmed >= 3000
                        <i class="fa fa-circle" style="font-size:14px;color:#660000"></i>
        </div>
        '''   

In [25]:
m = folium.Map([36, 128],width='100%', height='100%', zoom_start=7) 

plugins.Fullscreen(position='topright',  # Full screen
                   title='Click to Expand', 
                   title_cancel='Click to Exit', 
                   force_separate_button=True).add_to(m)

plugins.HeatMapWithTime(data, index=time_index, name=None, 
                               radius=25, min_opacity=0.3, max_opacity=0.6, 
                               scale_radius=False, gradient=None, 
                               use_local_extrema=False, auto_play=True, 
                               display_index=True, index_steps=1, min_speed=0.1, 
                               max_speed=10, speed_step=0.1, position='bottomleft', 
                               overlay=True, control=True, show=True).add_to(m)


    m.get_root().html.add_child(folium.Element(name))
  
plugins.MousePosition().add_to(m) ## you can easily get coordinates.
m

In [23]:
heat_data = regional_patient\
.groupby(['latitude','longitude'])['patient_id'].count().reset_index()\
.values\
.tolist()

m = folium.Map([36, 128], zoom_start=7)#, tiles = 'stamentoner')

folium.plugins.HeatMap(heat_data).add_to(m)

m